In [1]:
# For Google Colab
from google.colab import drive
drive.mount('/content/drive')
import sys
import os
sys.path.append('/content/drive/My Drive/c147-project')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

from models.cnn_lstm import *
from scripts.preprocessing import *

In [3]:
# Load Data
%cd /content/drive/My Drive/c147-project/project_data/project
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

## Adjusting the labels so that

# Cue onset left - 0
# Cue onset right - 1
# Cue onset foot - 2
# Cue onset tongue - 3

y_train_valid -= 769
y_test -= 769

/content/drive/My Drive/c147-project/project_data/project


In [4]:
## Data Augumentation
def aug(X_train, y_train, X_test, y_test, Trim, sub_sample, average, noise, noid):

    total_X_train = None
    total_y_train = None
    total_X_test = None
    total_y_test = None

    #Trimming
    H = int(Trim * X_train.shape[2])

    X_train = X_train[:, :, 0:H]

    X_test = X_test[:,:, 0:H]

    #Maxpooling
    X_max_train = np.max(X_train.reshape(X_train.shape[0], X_train.shape[1], -1, sub_sample), axis=3)

    X_max_test = np.max(X_test.reshape(X_test.shape[0], X_test.shape[1], -1, sub_sample), axis=3)

    total_X_train = X_max_train
    total_y_train = y_train

    total_X_test = X_max_test
    total_y_test = y_test

    #Jittering
    X_average_train = np.mean(X_train.reshape(X_train.shape[0], X_train.shape[1], -1, average),axis=3)
    X_average_test = np.mean(X_test.reshape(X_test.shape[0], X_test.shape[1], -1, average),axis=3)

    X_average_train = X_average_train + np.random.normal(0.0, noise, X_average_train.shape)
    X_average_test = X_average_test + np.random.normal(0.0, noise, X_average_test.shape)

    total_X_train = np.vstack((total_X_train, X_average_train))
    total_y_train = np.hstack((total_y_train, y_train))

    total_X_test = np.vstack((total_X_test, X_average_test))
    total_y_test = np.hstack((total_y_test, y_test))

    #Subsampling
    for i in range(sub_sample):

        X_subsample_train = X_train[:, :, i::sub_sample] + \
                            (np.random.normal(0.0, noise, X_train[:, :,i::sub_sample].shape) if noid else 0.0)

        total_X_train = np.vstack((total_X_train, X_subsample_train))
        total_y_train = np.hstack((total_y_train, y_train))

        X_subsample_test = X_test[:, :, i::sub_sample] + \
                            (np.random.normal(0.0, noise, X_test[:, :,i::sub_sample].shape) if noid else 0.0)

        total_X_test = np.vstack((total_X_test, X_subsample_test))
        total_y_test = np.hstack((total_y_test, y_test))


    return total_X_train,total_y_train,total_X_test,total_y_test

In [5]:
##Preprocessing
# Augmentation
X_train_valid_prep, y_train_valid_prep, X_test_prep, y_test_prep = aug(X_train_valid,y_train_valid,X_test,y_test,700,2,2,0.5,1)
# Splitting data
X_train_prep, X_valid_prep, y_train_prep, y_valid_prep = train_test_split(X_train_valid_prep, y_train_valid_prep, test_size=0.2, shuffle=True)

print((X_train_prep).shape)
print((X_valid_prep).shape)
print((X_test_prep).shape)

class EEGDataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.FloatTensor(X).unsqueeze(1)
        # self.X = self.X.permute(0, 3, 2, 1)
        self.Y = torch.LongTensor(Y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.Y[index]

    def shape(self):
        return self.X.shape

# Creating Dataset instances
train_set = EEGDataset(X_train_prep, y_train_prep)
val_set = EEGDataset(X_valid_prep, y_valid_prep)
test_set = EEGDataset(X_test_prep, y_test_prep)

print(train_set.shape())
print(val_set.shape())
print(test_set.shape())

# Initializing DataLoaders with concise batch_size and shuffle parameters
train_loader = DataLoader(train_set, batch_size=100, shuffle=True)
val_loader = DataLoader(val_set, batch_size=100, shuffle=True)
test_loader = DataLoader(test_set, batch_size=50, shuffle=True)

(6768, 22, 500)
(1692, 22, 500)
(1772, 22, 500)
torch.Size([6768, 1, 22, 500])
torch.Size([1692, 1, 22, 500])
torch.Size([1772, 1, 22, 500])


In [6]:
model = CNNLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [ ]:
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}, Validation Loss: {val_loss/len(val_loader)}, Validation Accuracy: {100 * correct / total}%")
